In [1]:
import grpc
import sys
import os
import chromadb
from sentence_transformers import SentenceTransformer

# ==========================================
# 1. استيراد ملفات البروتوكول المولدة
# ==========================================
# نضيف المسار الحالي ليتعرف بايثون على الملفات
sys.path.append(os.getcwd())

try:
    import rag_pb2 as pb2
    import rag_pb2_grpc as pb2_grpc
    print("✅ تم استيراد مكتبات البروتوكول (rag_pb2) بنجاح.")
except ImportError:
    print("❌ خطأ: لم يتم العثور على ملفات rag_pb2.py أو rag_pb2_grpc.py")
    print("تأكد أنك قمت بتشغيل أمر توليد البروتوكول في التيرمنال:")
    print("python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. rag.proto")

# إعدادات الاتصال بالسيرفر
GRPC_PORT = "50052"
SERVER_ADDRESS = f'localhost:{GRPC_PORT}'
print(f"🔗 إعداد الاتصال على: {SERVER_ADDRESS}")

/home/ahmed/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ تم استيراد مكتبات البروتوكول (rag_pb2) بنجاح.
🔗 إعداد الاتصال على: localhost:50052


In [2]:
# ==========================================
# 2. ربط قاعدة البيانات المحلية
# ==========================================
DB_PATH = "./chroma_db_storage"
COLLECTION_NAME = "medical_knowledge_base"
EMBEDDING_MODEL = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'

print("⏳ جاري تحميل قاعدة البيانات ونموذج التضمين...")

try:
    # تحميل العميل والقاعدة
    chroma_client = chromadb.PersistentClient(path=DB_PATH)
    collection = chroma_client.get_collection(name=COLLECTION_NAME)
    
    # تحميل نموذج التضمين (لتحويل السؤال لأرقام)
    # embed_model = SentenceTransformer(EMBEDDING_MODEL)
    embed_model = SentenceTransformer(EMBEDDING_MODEL, device="cpu")

    print("✅ قاعدة البيانات جاهزة للبحث.")
except Exception as e:
    print(f"❌ خطأ في تحميل قاعدة البيانات: {e}")
    print("تأكد أنك قمت بتشغيل ملف 02_Build_Vector_DB.ipynb أولاً.")

⏳ جاري تحميل قاعدة البيانات ونموذج التضمين...
✅ قاعدة البيانات جاهزة للبحث.


In [3]:
# ==========================================
# 3. دالة الاتصال بالسيرفر (Client Logic)
# ==========================================
async def get_ai_response(chat_history, max_tokens=2048, temperature=0.5):
    """
    تقوم بإرسال الرسائل لسيرفر gRPC وتطبع الرد مباشرة (Streaming)
    """
    # إنشاء قناة اتصال
    async with grpc.aio.insecure_channel(SERVER_ADDRESS) as channel:
        # تعريف الـ Stub (الواجهة البرمجية)
        stub = pb2_grpc.MedicalChatServiceStub(channel)
        
        # 1. تحويل الرسائل (List of Dicts) إلى (Proto Messages)
        proto_messages = []
        for msg in chat_history:
            proto_messages.append(pb2.Message(
                role=msg['role'],
                content=msg['content']
            ))
            
        # 2. إعدادات التوليد
        config = pb2.GenerationConfig(
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=0.95
        )
        
        # 3. بناء الطلب
        request = pb2.ChatRequest(
            session_id="test-session-01",
            messages=proto_messages,
            config=config
        )
        
        print("🤖 جاري استقبال الرد من السيرفر...\n")
        print("-" * 50)
        
        full_response = ""
        
        try:
            # 4. استقبال البث (Stream)
            async for response in stub.GenerateStream(request):
                # طباعة الرمز (Token) فور وصوله
                print(response.token, end="", flush=True)
                full_response += response.token
                
        except grpc.RpcError as e:
            print(f"\n❌ فشل الاتصال بالسيرفر: {e.code()}")
            print(f"التفاصيل: {e.details()}")
            return None
            
        print("\n" + "-" * 50)
        return full_response

In [4]:
# ==========================================
# 4. دالة البحث (Retrieval)
# ==========================================
def retrieve_context(query, n_results=4):
    # تحويل السؤال لأرقام
    query_vec = embed_model.encode([query]).tolist()
    
    # البحث في ChromaDB
    results = collection.query(
        query_embeddings=query_vec,
        n_results=n_results
    )
    
    # تنسيق النتائج كسياق نصي
    context_str = ""
    if results['documents']:
        for i, doc in enumerate(results['documents'][0]):
            meta = results['metadatas'][0][i]
            context_str += f"--- معلومة رقم {i+1} ---\n"
            context_str += f"السؤال: {meta['original_question']}\n"
            context_str += f"الجواب: {meta['answer']}\n\n"
            
    return context_str

In [5]:
# ==========================================
# 5. اختبار النظام المتكامل
# ==========================================

# 1. اكتب سؤالك هنا (يجب أن يكون من مجالك الطبي)
user_query = "كيف يتم تشخيص مرض السكري وما هي طرق علاجه؟"

print(f"🔍 السؤال: {user_query}")
print("⚙️ جاري البحث عن مصادر...")

# 2. جلب السياق
context = retrieve_context(user_query)

if not context:
    print("⚠️ تنبيه: لم يتم العثور على سياق مطابق في قاعدة البيانات.")
    context = "لا تتوفر معلومات مرجعية."
else:
    print("✅ تم العثور على سياق طبي داعم.")

# 3. تجهيز الرسالة للذكاء الاصطناعي
# تعليمات النظام (System Prompt) مهمة جداً لتوجيه النموذج
system_instruction = """
أنت مساعد طبي متخصص ومحترف. 
اعتمد بشكل أساسي على "السياق الطبي المرجعي" في إجابتك.
تحدث باللغة العربية الفصحى بأسلوب مهذب وواضح.
إذا لم تجد الإجابة في السياق، أخبر المستخدم بذلك بصدق.
"""

messages = [
    {"role": "system", "content": system_instruction},
    {"role": "user", "content": f"السياق الطبي المرجعي:\n{context}\n\nسؤال المستخدم:\n{user_query}"}
]

# 4. الإرسال (await ضرورية لأننا في Jupyter)
final_response = await get_ai_response(messages, max_tokens=8192)

🔍 السؤال: كيف يتم تشخيص مرض السكري وما هي طرق علاجه؟
⚙️ جاري البحث عن مصادر...
✅ تم العثور على سياق طبي داعم.
🤖 جاري استقبال الرد من السيرفر...

--------------------------------------------------
Alright, I need to answer the user's question in Arabic about how diabetes is diagnosed and its treatments. Let me check the reference medical context provided.

First, looking at the reference materials:

**معلومة رقم 1 (Fact 1)** talks about how diabetes starts and types. It mentions that diagnosis involves tests like random blood sugar test, fasting blood sugar test, oral glucose tolerance test. Also differentiates between Type 1 and Type 2 diabetes.

**معلومة رقم 2 (Fact 2)** covers the types of diabetes and treatment methods. It says there are two main types: Type 1 (no insulin production) treated with insulin injections; Type 2 (insulin resistance or reduced production) treated with medication, lifestyle changes, diet, exercise, self-care monitoring.

**معلومة رقم 3 (Fact 3)** is about p

In [7]:
from IPython.display import display, HTML

display(HTML(f"""
<div style="
    max-height: 600px;
    overflow-y: auto;
    white-space: pre-wrap;
    border: 1px solid #ccc;
    padding: 10px;
    border-radius: 8px;
    font-family: Arial;
">
{final_response}
</div>
"""))
